# PageSpeed Insightsのスコア取得

## ライブラリの呼び出し

In [1]:
import pandas as pd 
import sys,json,os,datetime 
from dotenv import load_dotenv, find_dotenv # 環境変数の取得用
import urllib.request # url取得用
from urllib.parse import urlparse # ドメイン取得用

## 環境変数の呼び出し

In [2]:
#  ディレクトリから.envファイルを探す
dotenv_path = find_dotenv()

# 環境変数をロード
load_dotenv(dotenv_path)

# API_KEYに環境変数を代入
API_KEY = os.environ.get("PageSpeed_API_KEY")

## urllist.csvから、url一覧を取得

In [3]:
url_list = []
with open('urllist.csv','r') as f:
    for row in f:
        url_list.append(row.rstrip('\n'))

## 格納用のリスト・変数の準備

In [4]:
# 結果格納用のリスト
output = []

# デバイス情報を格納
strategy = ['desktop','mobile']

# データ取得日を格納
date = datetime.date.today().strftime('%Y/%m/%d')

## APIをコールし、レスポンスを取得する関数

In [5]:
def api_to_list(url,device):

    # 結果格納用のリスト定義
    single_list = [] 
    
    api_format = 'https://www.googleapis.com/pagespeedonline/v2/runPagespeed?url={}&strategy={}&key={}'
    api = api_format.format(url,device,API_KEY)
    response = urllib.request.urlopen(api)
    data = json.loads(response.read())
    
    # URLをパースする
    parsed_url = urlparse(url)

    # ドメインの取得
    domain = parsed_url.netloc
    
    # レスポンスコードが200の場合はページスピードスコアを、それ以外はerrorを格納
    if data['responseCode'] == 200:
        title = data['title'] # ページtitleの取得
        score = data['ruleGroups']['SPEED']['score'] # ページスピードスコアの取得
        stats = data['pageStats'] # pageStatsの取得
    else:
        score = 'error' 
        stats = {
            "numberResources": 'error' ,
            "numberHosts": 'error' ,
            "totalRequestBytes": 'error' ,
            "numberStaticResources": 'error' ,
            "htmlResponseBytes": 'error' ,
            "textResponseBytes": 'error' ,
            "cssResponseBytes": 'error' ,
            "imageResponseBytes": 'error',
            "javascriptResponseBytes": 'error',
            "otherResponseBytes": 'error',
            "numberJsResources": 'error',
            "numberCssResources": 'error'
        }
    
    # リストへの格納
    single_list = [domain,date,device,url,title,score]
    for value in stats.values():
        single_list.append(value)

    return(single_list)

## APIをrequestし、結果をoutputに格納する関数

In [6]:
def get_list(url_list,strategy):
    for url in url_list:
        for device in strategy:
            output.append(api_to_list(url,device))
    return output

## 関数の実行結果をresultに格納

In [7]:
result = get_list(url_list,strategy)

## 実行結果をデータフレームに変換

In [8]:
df = pd.DataFrame(
    result,
    columns=['ドメイン','データ取得日','デバイス','ページURL','ページtitle','ページスピードスコア','numberResources',
                   'numberHosts','totalRequestBytes','numberStaticResources','htmlResponseBytes','textResponseBytes','cssResponseBytes',
                   'imageResponseBytes','javascriptResponseBytes','otherResponseBytes','numberJsResources','numberCssResources']
     )
df

,ドメイン,データ取得日,デバイス,ページURL,ページtitle,ページスピードスコア,numberResources,numberHosts,totalRequestBytes,numberStaticResources,htmlResponseBytes,textResponseBytes,cssResponseBytes,imageResponseBytes,javascriptResponseBytes,otherResponseBytes,numberJsResources,numberCssResources
0,employment.en-japan.com,2018/01/20,desktop,https://employment.en-japan.com/,転職サイトなら日本最大級の【エン転職】,44,247,78,53119,121,91025,242,237614,2027758,1010657,116633,60,7
1,employment.en-japan.com,2018/01/20,mobile,https://employment.en-japan.com/,エン転職,40,247,86,55175,91,103162,242,369865,2101158,1059878,36949,60,2
2,mid-tenshoku.com,2018/01/20,desktop,https://mid-tenshoku.com/,転職｜30代、40代の転職サイトは【ミドルの転職】エンジャパン,50,124,51,25698,69,115726,242,219761,1482182,1213116,30018,32,2
3,mid-tenshoku.com,2018/01/20,mobile,https://mid-tenshoku.com/,転職｜30代、40代の転職サイトは【ミドルの転職】エンジャパン,40,69,41,16487,32,34133,242,236796,2488346,1211969,23509,32,2


## csvに出力

In [9]:
df.to_csv("result.csv")